# 1) Importation des librairies

In [42]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

# Pour la séparation des données en train/test
from sklearn.model_selection import train_test_split

# Pour la création et l'évaluation de modèles
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score

# 2) Chargement des données

In [43]:
# Lis les différents fichiers CSV
folder = "src/"
df_general = pd.read_csv(folder + "general_data.csv")
df_emp_survey = pd.read_csv(folder + "employee_survey_data.csv")
df_mgr_survey = pd.read_csv(folder + "manager_survey_data.csv")
df_in_time = pd.read_csv(folder + "in_time.csv")
df_out_time = pd.read_csv(folder + "out_time.csv")

# 3) Exploration des données

In [44]:
print("=== General Data ===")
print(df_general.head(), "\n")
print(df_general.info(), "\n")

print("=== Employee Survey Data ===")
print(df_emp_survey.head(), "\n")
print(df_emp_survey.info(), "\n")

print("=== Manager Survey Data ===")
print(df_mgr_survey.head(), "\n")
print(df_mgr_survey.info(), "\n")

print("=== In Time Data ===")
print(df_in_time.head(), "\n")
print(df_in_time.info(), "\n")

print("=== Out Time Data ===")
print(df_out_time.head(), "\n")
print(df_out_time.info(), "\n")

=== General Data ===
   Age Attrition     BusinessTravel              Department  DistanceFromHome  \
0   51        No      Travel_Rarely                   Sales                 6   
1   31       Yes  Travel_Frequently  Research & Development                10   
2   32        No  Travel_Frequently  Research & Development                17   
3   38        No         Non-Travel  Research & Development                 2   
4   32        No      Travel_Rarely  Research & Development                10   

   Education EducationField  EmployeeCount  EmployeeID  Gender  ...  \
0          2  Life Sciences              1           1  Female  ...   
1          1  Life Sciences              1           2  Female  ...   
2          4          Other              1           3    Male  ...   
3          5  Life Sciences              1           4    Male  ...   
4          1        Medical              1           5    Male  ...   

   NumCompaniesWorked Over18 PercentSalaryHike  StandardHours  \


 # 4) Fusion des données

## 4.1) Merge general_data, employee_survey_data et manager_survey_data

In [45]:
df_merged = pd.merge(df_general, df_emp_survey, on="EmployeeID")
df_merged = pd.merge(df_merged, df_mgr_survey, on="EmployeeID")

## 4.2) Calculer le nombre d'heures travaillées par jour
On va aussi créer des features à partir de df_in_time et df_out_time (par exemple : nombre d'heures travaillées moyennes). </br>
Les colonnes de in_time / out_time sont des dates/heures pour chaque jour travaillé.

In [46]:
#  - Pour chaque employé, on peut calculer la différence out_time - in_time (en heures) pour chaque jour.
#  - Ensuite on peut faire la moyenne sur toutes les dates disponibles pour obtenir "moyenne d'heures/jour"

# Replace Unnamed header by EmployeeID
df_in_time.rename(columns={"Unnamed: 0": "EmployeeID"}, inplace=True)
df_out_time.rename(columns={"Unnamed: 0": "EmployeeID"}, inplace=True)

# Suppression de la première colonne "EmployeeID" pour faciliter les opérations (on la conserve à part)
df_in_time_id = df_in_time['EmployeeID']
df_out_time_id = df_out_time['EmployeeID']

# On exclut la colonne 'EmployeeID' des dataframes pour ne traiter que les colonnes date/heure
df_in_time_dates = df_in_time.drop(['EmployeeID'], axis=1)
df_out_time_dates = df_out_time.drop(['EmployeeID'], axis=1)

# Conversion des valeurs string en datetime pour permettre la soustraction des temps
# Nota : Certaines valeurs sont "NA" => conversion en NaT
df_in_time_dates = df_in_time_dates.apply(pd.to_datetime, errors='coerce')
df_out_time_dates = df_out_time_dates.apply(pd.to_datetime, errors='coerce')

# Calcul de la différence (out_time - in_time) par employé et par jour
df_hours = df_out_time_dates - df_in_time_dates  # Résultat en format timedelta

# Convertir les timedelta en nombre d'heures (float)
df_hours = df_hours.apply(lambda x: x.dt.total_seconds() / 3600)

# Exemple : Calcul d'une statistique agrégée (moyenne d'heures/jour travaillé) pour chaque employé
df_hours['mean_work_hours'] = df_hours.mean(axis=1)

# On peut aussi calculer le nombre de jours d'absence (journées entières manquantes => in_time = NA & out_time = NA)
# ou le ratio de jours travaillés vs le total possible, etc.
# Ci-dessous un exemple de calcul du nombre de jours (colonnes) pour lesquels l'entrée est manquante
nb_jours_total = df_hours.shape[1] - 1  # -1 car la dernière colonne est 'mean_work_hours' qu'on vient d'ajouter
df_hours['absent_days'] = df_hours.iloc[:, :-1].isna().sum(axis=1)  # On ne compte pas la col. 'mean_work_hours'

# Concaténer EmployeeID pour pouvoir refusionner
df_hours_final = pd.concat([df_in_time_id, df_hours[['mean_work_hours','absent_days']]], axis=1)

## 4.3) Merge avec df_merged pour rajouter ces nouvelles features

In [47]:
df_merged = pd.merge(df_merged, df_hours_final, on='EmployeeID', how='left')

print("\n=== Aperçu des données fusionnées ===\n")
print(df_merged.head())
print(df_merged.info())


=== Aperçu des données fusionnées ===

   Age Attrition     BusinessTravel              Department  DistanceFromHome  \
0   51        No      Travel_Rarely                   Sales                 6   
1   31       Yes  Travel_Frequently  Research & Development                10   
2   32        No  Travel_Frequently  Research & Development                17   
3   38        No         Non-Travel  Research & Development                 2   
4   32        No      Travel_Rarely  Research & Development                10   

   Education EducationField  EmployeeCount  EmployeeID  Gender  ...  \
0          2  Life Sciences              1           1  Female  ...   
1          1  Life Sciences              1           2  Female  ...   
2          4          Other              1           3    Male  ...   
3          5  Life Sciences              1           4    Male  ...   
4          1        Medical              1           5    Male  ...   

   YearsAtCompany YearsSinceLastPromotion Year

# 5) Nettoyage et préparation des données

## 5.1) Gérer les valeurs manquantes

In [48]:
# On regarde déjà combien de valeurs manquantes par colonne
missing_values = df_merged.isnull().sum()
print("\n=== Nombre de valeurs manquantes par colonne ===\n", missing_values)


=== Nombre de valeurs manquantes par colonne ===
 Age                         0
Attrition                   0
BusinessTravel              0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
EmployeeID                  0
Gender                      0
JobLevel                    0
JobRole                     0
MaritalStatus               0
MonthlyIncome               0
NumCompaniesWorked         19
Over18                      0
PercentSalaryHike           0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           9
TrainingTimesLastYear       0
YearsAtCompany              0
YearsSinceLastPromotion     0
YearsWithCurrManager        0
EnvironmentSatisfaction    25
JobSatisfaction            20
WorkLifeBalance            38
JobInvolvement              0
PerformanceRating           0
mean_work_hours             0
absent_days                 0
dtype: int64


In [49]:
# Dropna sur la colonne Attrition car c'est notre target
df_merged = df_merged.dropna(subset=['Attrition'])

In [56]:
# Exemple d'imputation : pour 'mean_work_hours' et 'absent_days', on remplace les NaN par la moyenne
# On privilégie la moyenne ici car les valeurs de type durée (heures) ne sont pas fortement bornées
df_merged['mean_work_hours'] = df_merged['mean_work_hours'].fillna(df_merged['mean_work_hours'].mean())
print(f"Filled missing values for column 'mean_work_hours' with mean value {df_merged['mean_work_hours'].mean()}")
df_merged['absent_days'] = df_merged['absent_days'].fillna(df_merged['absent_days'].mean())
print(f"Filled missing values for column 'absent_days' with mean value {df_merged['absent_days'].mean()}")

Filled missing values for column 'mean_work_hours' with mean value 7.700791784478147
Filled missing values for column 'absent_days' with mean value 24.73469387755102


In [59]:
# Pour les colonnes de satisfaction ou autres colonnes numériques manquantes, on peut aussi faire un fillna
# On privilégie la médiane
num_cols = ['EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance', 
            'JobInvolvement', 'PerformanceRating', 'TotalWorkingYears', 'NumCompaniesWorked',]
for col in num_cols:
    if col in df_merged.columns:
        df_merged[col] = df_merged[col].fillna(df_merged[col].median())
        print(f"Filled missing values for column {col} with median value {df_merged[col].median()}")

Filled missing values for column EnvironmentSatisfaction with median value 3.0
Filled missing values for column JobSatisfaction with median value 3.0
Filled missing values for column WorkLifeBalance with median value 3.0
Filled missing values for column JobInvolvement with median value 3.0
Filled missing values for column PerformanceRating with median value 3.0
Filled missing values for column TotalWorkingYears with median value 10.0
Filled missing values for column NumCompaniesWorked with median value 2.0


## 5.2) Nettoyage de certaines colonnes (ex: Over18, EmployeeCount, StandardHours)

In [52]:
# On remarque souvent dans ce dataset "EmployeeCount" est toujours 1 => pas d'intérêt
# "Over18" est toujours "Y", "StandardHours" est souvent 8 => on peut les drop
cols_to_drop = ['Over18','StandardHours','EmployeeCount'] 
for c in cols_to_drop:
    if c in df_merged.columns:
        df_merged.drop(c, axis=1, inplace=True, errors='ignore')

## 5.3) Conversion de colonnes catégorielles en numériques

In [54]:
# Par exemple, Attrition = Yes/No, Gender = Male/Female, etc.
# On peut les encoder, soit via LabelEncoder, soit via OneHotEncoder
# Commençons par un label encoding simple pour la variable cible

df_merged['Attrition'] = df_merged['Attrition'].map({'Yes':1, 'No':0})

# Autres colonnes catégorielles (BusinessTravel, Department, EducationField, Gender, MaritalStatus, JobRole...)
cat_cols = ['BusinessTravel','Department','EducationField','Gender','MaritalStatus','JobRole']

# On va faire un one-hot-encoding rapide:
df_merged = pd.get_dummies(df_merged, columns=cat_cols, drop_first=True)

KeyError: "None of [Index(['BusinessTravel', 'Department', 'EducationField', 'Gender',\n       'MaritalStatus', 'JobRole'],\n      dtype='object')] are in the [columns]"

# 6) Séparation des données en train/test

In [30]:
# On sépare la cible (Attrition) des features

X = df_merged.drop(['EmployeeID','Attrition'], axis=1)
y = df_merged['Attrition']

# Ensuite on fait un split train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# 7) scaling et entraînement d'un modèle simple

In [31]:
# Selon le modèle (Logistic Regression par exemple), il peut être intéressant de standardiser
# Ici on va montrer un exemple de pipeline manuel (scaling + logistic regression).

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## 7.1) Entrainement d'un modèle de Logistic Regression

In [38]:
#clf_lr = LogisticRegression(random_state=42, max_iter=500)
#clf_lr.fit(X_train_scaled, y_train)

Age                                   0
DistanceFromHome                      0
Education                             0
JobLevel                              0
MonthlyIncome                         0
NumCompaniesWorked                   15
PercentSalaryHike                     0
StockOptionLevel                      0
TotalWorkingYears                     0
TrainingTimesLastYear                 0
YearsAtCompany                        0
YearsSinceLastPromotion               0
YearsWithCurrManager                  0
EnvironmentSatisfaction               0
JobSatisfaction                       0
WorkLifeBalance                       0
JobInvolvement                        0
PerformanceRating                     0
mean_work_hours                       0
absent_days                           0
BusinessTravel_Travel_Frequently      0
BusinessTravel_Travel_Rarely          0
Department_Research & Development     0
Department_Sales                      0
EducationField_Life Sciences          0
